##  Creating the DB
this document is solely for testing localy, i am not sure if it can be applied to the cirrent code but i will try my best to make it as compatible as possible

In [ ]:
#### imports needed
import sqlite3 
import os
import pandas as pd

In [1]:
def creatingDB(file_path, DB_name) :
  # Load CSV
  df = pd.read_csv(file_path)

  # Connect to database
  conn = sqlite3.connect(DB_name)

  # Automatically create table with same name and structure
  df.to_sql(DB_name, conn, if_exists='append', index=False)

  # Done
  conn.commit()
  conn.close()

  print("Table created with the same structure as the CSV and data inserted.")


In [ ]:
def compute_metrics(conn, pred_table, gt_table, compare_column, join_key):
    cursor = conn.cursor()

    # --- True Positives (correct predictions) ---
    cursor.execute(f'''
        SELECT COUNT(*) FROM {pred_table} p
        INNER JOIN {gt_table} g ON p.{join_key} = g.{join_key}
        WHERE p.{compare_column} = g.{compare_column}
    ''')
    TP = cursor.fetchone()[0]

    # --- False Positives (predicted wrong) ---
    cursor.execute(f'''
        SELECT COUNT(*) FROM {pred_table} p
        INNER JOIN {gt_table} g ON p.{join_key} = g.{join_key}
        WHERE p.{compare_column} != g.{compare_column}
    ''')
    FP = cursor.fetchone()[0]

    # --- False Negatives (missed predictions) ---
    # rows in GT that don't exist in predictions (e.g., no prediction at all)
    cursor.execute(f'''
        SELECT COUNT(*) FROM {gt_table} g
        LEFT JOIN {pred_table} p ON g.{join_key} = p.{join_key}
        WHERE p.{compare_column} IS NULL
    ''')
    FN = cursor.fetchone()[0]

    # --- Metrics ---
    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'Accuracy': round(accuracy, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'F1': round(f1, 4)
    }


Populating the data base

In [ ]:
excel_file = 'data.xlsx'
df = pd.read_excel(excel_file)
#selecting only the used columns
usedColumns = df[['Study_ID', 'Study', 'Allocation', 'Experimenter']]

for study in usedColumns[['Study']]:
  format(study)

conn = sqlite3.connect('evaluation.db')
cursor = conn.cursor()

usedColumns.to_sql('Ground_truth', conn, if_exists='append', index=False)

conn.commit()
conn.close()

Trying to fill the model database

In [ ]:

data_frames = []
csv_directory = 'extracted'

if os.path.exists(csv_directory):
  for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
      file_path = os.path.join(csv_directory, filename)
      try:
        df = pd.read_csv(file_path)
        data_frames.append(df)
        print(f"Successfully read {filename}")
      except Exception as e:
        print(f"Error reading {filename}: {e}")

if data_frames:
  # Concatenate all the dataframes into one
  combined_df = pd.concat(data_frames, ignore_index=True)
  print("\nCombined DataFrame:")
  print(combined_df)
else:
  print("No CSV files found in the directory.")

Successfully read Korat and Shamir - 2007 - Electronic books versus adult readers effects on children's emergent literacy as a function of soci.pdf.csv
Successfully read Blom-Hoffman et al. - 2007 - Instructing Parents to Use Dialogic Reading Strategies with Preschool Children Impact of a Video-Ba.pdf.csv
Successfully read Coyne et al. - 2004 - Teaching Vocabulary During Shared Storybook Readings An Examination of Differential Effects.pdf.csv
Successfully read Fontes and Cardoso-Martins - 2004 - Efeitos da leitura de histórias no desenvolvimento da linguagem de crianças de nível sócio-econômico.pdf.csv
Successfully read Segal-Drori et al. - 2010 - Reading electronic and printed books with and without adult instruction effects on emergent reading.pdf.csv
Successfully read Coyne et al. - 2010 - Direct and Extended Vocabulary Instruction in Kindergarten Investigating Transfer Effects.pdf.csv
Successfully read Aram and Biron - 2004 - Joint storybook reading and joint writing interventions 

In [ ]:
conn = sqlite3.connect('evaluation.db')
cursor = conn.cursor()

# Populate the Model table with combined_df
combined_df.to_sql('Model', conn, if_exists='append', index=False)

conn.commit()
conn.close()

print("\nModel table populated with combined_df.")

# Optional: Verify the data in the Model table
conn = sqlite3.connect('extracted.db')
#model_df = pd.read_sql_query("SELECT * FROM Model", conn)
conn.close()

print("\nData in Model table:")
model_df


Model table populated with combined_df.

Data in Model table:


,Allocation,Experimenter,Study
0,Random,Teacher,Korat and Shamir - 2007 - Electronic books ver...
1,Random,Parent,Instructing Parents to Use Dialogic Reading St...
2,Random,Researcher,Coyne et al. 2004 - Teaching Vocabulary During...
3,Random,Researcher,Efeitos da Leitura de Histórias no Desenvolvim...
4,Random,Researcher,Segal-Drori et al. - 2010 - Reading electronic...
5,Random,Teacher,Coyne et al. 2010 - Direct and Extended Vocabu...
6,Random,Teacher,Joint storybook reading and joint writing inte...
7,Random,Researcher,Lonigan et al. - 1999 - Effects of Two Shared-...
8,Random,Parent,Promoting Language and Literacy Development th...
9,Random,Researcher,Purpura et al. 2017


The name of the studies are not ok on the doc extracted, idk how to fix it, ineed to look into anna's code in more details `(*>﹏<*)′

In [ ]:
conn = sqlite3.connect('evaluation.db')
conn2 = sqlite3.connect('extracted.db')
cursor = conn.cursor()
cursor2 = conn2.cursor()

tp = conn.execute('''
  SELECT COUNT(Model.Study)
  FROM Model JOIN Ground_truth ON Model.Study = Ground_truth.Study
  WHERE Model.Allocation = 'Random' AND Ground_truth.Allocation = 'Random'
''').fetchall()[0]


print(tp)

compute_metrics(conn, 'evaluation.db', 'evaluated.db', 'Allocation', 'Study')
conn.close()

(0,)


OperationalError: no such table: evaluation.db